## 1) 학습 방향 로드맵 (Pandas 숙련 목표)

1. **데이터 이해 & 검증(EDA 초반)**
    - shape, dtypes, 결측치 분포, 중복/유니크, 범주 분포 확인
    - “이 컬럼은 무엇을 의미하는가?”를 코드로 확인하는 습관
        
2. **정제/전처리(실무 핵심)**
    - 결측치 처리(단순 대체/그룹 기반 대체), 이상치 탐지
    - 문자열 파싱(Name/Title/Cabin), 카테고리 인코딩, 파생 변수
        
3. **집계 & 비교 분석**
    - groupby/agg, pivot_table, crosstab
    - “생존률” 같은 비율 지표를 안정적으로 계산하는 법(분모, NaN, 타입)
        
4. **조인 & 데이터 모델링**
    - train/test 분리 상태에서 동일 변환 적용
    - 외부 테이블(예: Deck lookup, Title mapping)을 merge/join으로 결합
        
5. **재현 가능성 & 파이프라인화**
    - 함수화, 체인 방식, 중간 산출물 검증(assert)
    - 성능(메모리/속도) 감각: astype('category'), 벡터화 vs apply

In [2]:
# import
import csv
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from enum import Enum


# A. 로딩/점검/기초 EDA (기본 체력)

### A1. 데이터 로딩 및 스키마 리포트
- **문제:** `train.csv`를 읽고, 
(1) 행/열 수, 
(2) 각 컬럼 dtype, 
(3) 결측치 개수/비율
(4) 각 컬럼의 유니크 개수
을 한 번에 요약한 표를 만드세요. 
- **무엇을 묻는가:** `read_csv`, `shape`, `dtypes`, `isna().sum()`, 비율 계산, `DataFrame` 구성 능력.

In [3]:
# methods
def generate_enum(df: pd.DataFrame, enum_name="Cols"):
    lines = [f"class {enum_name}(Enum):"]
    for col in df.columns:
        lines.append(f"    {col} = '{col}'")
    return "\n".join(lines)

In [ ]:
base_path = (
    Path(os.getcwd()).parent.parent
    / "data"
    / "raw"
    / "kaggle"
    / "datasets"
    / "titanic"
    / "titanic"
)
train_csv_path = base_path / "train.csv"
df_train_raw = pd.read_csv(train_csv_path)
df_train = df_train_raw.__deepcopy__()

In [5]:
# print(generate_enum(df_train))
class Cols(Enum):
    PassengerId = "PassengerId"
    Survived = "Survived"
    Pclass = "Pclass"
    Name = "Name"
    Sex = "Sex"
    Age = "Age"
    SibSp = "SibSp"
    Parch = "Parch"
    Ticket = "Ticket"
    Fare = "Fare"
    Cabin = "Cabin"
    Embarked = "Embarked"

In [6]:
# shape
df_train.shape
# dtypes
df_train.dtypes

# na data frame
na_cnt = df_train.isna().sum()
na_ratio = na_cnt / len(df_train)

df_na = pd.DataFrame(
    {"NA_CNT": na_cnt, "NA_RATIO": na_ratio, "DATA_TYPE": df_train.dtypes}
)
df_na["NA_RATIO"] = (df_na["NA_RATIO"] * 100).round(3)
df_na = df_na.reset_index()
df_na = df_na.rename(columns={"index": "column"})
df_na

,column,NA_CNT,NA_RATIO,DATA_TYPE
0,PassengerId,0,0.000,int64
1,Survived,0,0.000,int64
2,Pclass,0,0.000,int64
3,Name,0,0.000,object
4,Sex,0,0.000,object
5,Age,177,19.865,float64
6,SibSp,0,0.000,int64
7,Parch,0,0.000,int64
8,Ticket,0,0.000,object
9,Fare,0,0.000,float64


### A2. 유니크/중복 점검
- **문제:** PassengerId가 유니크인지 확인하고, 중복이 있다면 중복 행을 반환하세요. 또한 Name 중복 빈도 상위 10개를 뽑으세요.
- **무엇을 묻는가:** `duplicated`, `value_counts`, 인덱싱/필터링.

In [7]:
# check PassengerId is unique
duplicated_id_cnt = int(df_train.duplicated(subset=[Cols.PassengerId.value]).sum())

In [8]:
distributing_name_regax = r"^(?P<surname>[^,]+),\s*(?P<title>[^.]+)\.\s*(?P<rest>.*)$"
format_maiden = r"\((?P<maiden>[^)]+)\)"
format_delete_parenthesis = r"\s*\([^)]*\)\s*"

df_train = df_train.__deepcopy__()
df_name_base = df_train[Cols.Name.value].str.extract(distributing_name_regax)
df_name_base["maiden_name"] = df_name_base["rest"].str.extract(format_maiden)
df_name_base["given_raw"] = (
    df_name_base["rest"]
    .str.replace(format_delete_parenthesis, " ", regex=True)
    .str.strip()
)
# first/middle 분해 (given_raw 기준)
tokens = df_name_base["given_raw"].str.split()
df_name_base["first_raw"] = tokens.str[0]
df_name_base["middle_raw"] = tokens.str[1:].str.join(" ")

# count 붙이기
parsed = pd.concat([df_train, df_name_base.drop(columns=["rest"])], axis=1)

In [9]:
parsed["surname"].value_counts().head(10).index.tolist()

['Andersson',
 'Sage',
 'Skoog',
 'Panula',
 'Carter',
 'Goodwin',
 'Johnson',
 'Rice',
 'Fortune',
 'Williams']

### A3. 범주형 분포 빠르게 보기
- **문제:** `Sex`, `Embarked`, `Pclass`의 분포를 “개수 + 비율”로 동시에 보여주는 요약 테이블을 각각 생성하세요.
- **무엇을 묻는가:** `value_counts(normalize=...)`, `concat`, 멀티 컬럼 요약 패턴.

In [10]:
target_cols = [Cols.Sex.value, Cols.Embarked.value, Cols.Pclass.value]

# count
s_count = df_train[target_cols[0]].value_counts(dropna=False)
# ratio
s_ratio = df_train[target_cols[0]].value_counts(normalize=True, dropna=False)

dist = pd.DataFrame({"count": s_count, "ratio": s_ratio})

dist

,count,ratio
Sex,,
male,577,0.647587
female,314,0.352413


In [16]:
cat_cols = ["Sex", "Pclass", "Embarked"]

rows = []
for col in cat_cols:
    vc = df_train[col].value_counts(dropna=False)
    ratio = df_train[col].value_counts(normalize=True, dropna=False) * 100
    tmp = pd.DataFrame(
        {
            "value": vc.index,
            "count": vc.values,
            "ratio": ratio.map(lambda x: f"{x:.2f}%"),
        }
    )
    tmp.insert(0, "col", col)  # 맨 앞에 컬럼명 넣기
    rows.append(tmp)

dist_long = pd.concat(rows, ignore_index=True)
dist_long.sort_values(by="count", ascending=False)

,col,value,count,ratio
5,Embarked,S,644,72.28%
0,Sex,male,577,64.76%
2,Pclass,3,491,55.11%
1,Sex,female,314,35.24%
3,Pclass,1,216,24.24%
4,Pclass,2,184,20.65%
6,Embarked,C,168,18.86%
7,Embarked,Q,77,8.64%
8,Embarked,NaN,2,0.22%
